In [1]:
## Add this directory to the path and load our functions
import sys
sys.path.append("../src/")

from evaluating_results import process_labels

import os
import polars as pl

## Load the labels and votes for the 00 config
labeled_00_df = process_labels.get_merged_labels_and_votes(config="00")
labeled_00_df = process_labels.get_majority_vote(labeled_00_df)

In [2]:
## Load the query results from the 02 config
query_results_02_df = pl.read_parquet(
    "../data_labeling/raw_results/query_results_config_02_full.parquet")

## Join the 02 config query results with the labeled results from the 00 config
joined_results = query_results_02_df.join(
    labeled_00_df, on=["reddit_name", "query_text"], how="left")

## Only keep the results that have yet to be labeled
joined_results = joined_results.filter(
    joined_results["relevance_rating"].is_null())

## Only keep the columns that are needed so that they match the columns
## expected in the labeling script
columns_to_keep = query_results_02_df.columns
unlabled_02_df = joined_results[columns_to_keep].clone()

## Save the results
unlabled_02_df.write_parquet("../data_labeling/raw_results/query_results_config_02.parquet")